# 加载和分析《第三次浪潮》

## 准备

In [1]:
%%time

from llama_index.core import SummaryIndex, Document
from llama_index.core import Settings

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

from milvus import default_server
from pymilvus import connections, utility

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core.llms import ChatMessage

CPU times: user 3.56 s, sys: 480 ms, total: 4.04 s
Wall time: 3.67 s


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f81ed8ad060>, id_func=<function default_id_func at 0x7f82b33f7010>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

# 启动milvus服务器

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

# Stop your milvus server
# default_server.stop()

default_server.listen_port

v2.3.5-lite
CPU times: user 3.15 s, sys: 324 ms, total: 3.48 s
Wall time: 6.59 s


19530

In [4]:
%%time

# 初始化全局 llm

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 119 ms, sys: 3.91 ms, total: 123 ms
Wall time: 124 ms


In [5]:
%%time

# 初始化全局 embedding 模型

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 424 ms, sys: 16 ms, total: 440 ms
Wall time: 441 ms


In [6]:
%%time

import logging
import sys

# 设置系统日志，便于设置level排查

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 318 µs, sys: 36 µs, total: 354 µs
Wall time: 358 µs


In [7]:
%%time
%%capture

# load documents
documents = SimpleDirectoryReader(input_files=["books2/第三次浪潮.pdf"]).load_data()
for doc in documents:
    print(doc.doc_id, doc.metadata['file_name'])

Ignoring wrong pointing object 1517 0 (offset 0)
Ignoring wrong pointing object 1523 0 (offset 0)
Ignoring wrong pointing object 1529 0 (offset 0)
Ignoring wrong pointing object 1535 0 (offset 0)
Ignoring wrong pointing object 1541 0 (offset 0)
Ignoring wrong pointing object 1547 0 (offset 0)
Ignoring wrong pointing object 1553 0 (offset 0)
Ignoring wrong pointing object 1559 0 (offset 0)
Ignoring wrong pointing object 1565 0 (offset 0)
Ignoring wrong pointing object 1571 0 (offset 0)
Ignoring wrong pointing object 1577 0 (offset 0)
Ignoring wrong pointing object 1583 0 (offset 0)
Ignoring wrong pointing object 1589 0 (offset 0)
Ignoring wrong pointing object 1595 0 (offset 0)
Ignoring wrong pointing object 1601 0 (offset 0)
Ignoring wrong pointing object 1607 0 (offset 0)
Ignoring wrong pointing object 1613 0 (offset 0)
Ignoring wrong pointing object 1619 0 (offset 0)
Ignoring wrong pointing object 1625 0 (offset 0)
Ignoring wrong pointing object 1631 0 (offset 0)
Ignoring wrong point

In [8]:
%%time

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 9fed255c090b401a9f69e42f9a24d113
Created new connection using: 9fed255c090b401a9f69e42f9a24d113
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection
Successfully created an index on collection: llamacollection
CPU times: user 16.3 s, sys: 627 ms, total: 17 s
Wall time: 1min 37s


## 基于基本嵌入问答

In [9]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("宗教有哪些作用")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.16

In [10]:
%%time

streaming_response = query_engine.query("第三次浪潮有哪些主要特征，列表说明")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
第三次浪潮的主要特征包括：

- **技术分散**：随着工业化的减少，技术变得更加分散，不再局限于大型工厂或城市中心。
- **知识民主化**：信息的流通更加自由，互联网和全球通信网络使得知识无国界，个体可以获取和分享更多资源。
- **个性化与创新**：鼓励个人表达和创新，反对大规模生产中的标准化，支持定制化产品和服务。
- **科技重建**：信息技术和通信技术的革新重塑了经济结构，推动向服务和知识驱动的经济转型。
- **时间价值**：对时间的理解更注重效率和即时性，强调快速决策和信息处理的重要性。
- **政治分散化**：权力从中央向地方分散，可能涉及新的决策机制，如社会中对少数群体声音

In [11]:
%%time

streaming_response = query_engine.query("文中提到的英国病，指的是啥")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
</noinput>
CPU times: user 276 ms, sys: 4.3 ms, total: 280 ms
Wall time: 19.7 s


## 基于rerank的问答

In [12]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 923 ms, sys: 659 ms, total: 1.58 s
Wall time: 803 ms


In [13]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("宗教有哪些作用")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
宗教的作用包括提供共同意识、秩序以及意义。它给予人们一种团结和归属感，通过教义和严格的行为规范，帮助信徒整理和理解复杂的世界。尽管有些教派的教义可能在外人看来不切实际，但它们的组织结构为信徒提供了一个稳定的框架来存储信息，从而减轻认知负荷。此外，宗教还能满足人们在情感上的需求，如在寂寞时给予温暖和关怀，通过牺牲个人自由（如放弃某些物质享受或遵循严格的规定）来换取精神上的满足。然而，必须指出的是，宗教中的某些行为规范可能具有限制性和压迫性，如体罚和强制劳动等。
CPU times: user 1.48 s, sys: 108 ms, total: 1.59 s
Wall time: 7.77 s


In [14]:
%%time

streaming_response = query_engine.query("第三次浪潮有哪些主要特征，列表说明")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
- 分散式生产
- 适当的规模
- 可再生的能源
- 疏散都市人口
- 在家中工作
- 自己生产自己消费
- 非集中化的生活特征
- 弹性的工作节奏和多样性
- 能源需求量小，市场竞争激烈
CPU times: user 691 ms, sys: 8.02 ms, total: 699 ms
Wall time: 3.82 s


In [15]:
%%time

streaming_response = query_engine.query("文中提到的英国病，指的是啥")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
文中提到的“英国病”是指在西方国家中普遍存在的对辛勤工作的漠视态度，尤其是管理人员对于人们工作努力程度下降的担忧。他们认为这种现象如果不加以纠正，可能会在未来变得更糟。这可能与工业社会的一些问题有关，如过度的物质主义和对健康、艺术和心理层面的影响，以及对疯人院病患的不正常看法。
CPU times: user 762 ms, sys: 16.1 ms, total: 778 ms
Wall time: 4.35 s
